In [30]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.metrics import Precision, Recall


In [2]:
# Load Nutrient Data
csv_path = 'indian_dishes_nutrients.csv'  # Ensure columns: 'Food', 'Calories', 'Protein', 'Fat', 'Carbs'
data = pd.read_csv(csv_path)
X_nutrients = data.drop(columns=['Food'])
y = data['Food']

In [3]:
# Encode Food Labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


In [4]:
# Load Image Data
data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
image_size = (128, 128)
batch_size = 32

train_images = data_gen.flow_from_directory(
    'Indian Food Images',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',
    subset='training'
)

val_images = data_gen.flow_from_directory(
    'Indian Food Images',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',
    subset='validation'
)

Found 3200 images belonging to 80 classes.
Found 800 images belonging to 80 classes.


In [5]:
train_images.class_indices

{'adhirasam': 0,
 'aloo_gobi': 1,
 'aloo_matar': 2,
 'aloo_methi': 3,
 'aloo_shimla_mirch': 4,
 'aloo_tikki': 5,
 'anarsa': 6,
 'ariselu': 7,
 'bandar_laddu': 8,
 'basundi': 9,
 'bhatura': 10,
 'bhindi_masala': 11,
 'biryani': 12,
 'boondi': 13,
 'butter_chicken': 14,
 'chak_hao_kheer': 15,
 'cham_cham': 16,
 'chana_masala': 17,
 'chapati': 18,
 'chhena_kheeri': 19,
 'chicken_razala': 20,
 'chicken_tikka': 21,
 'chicken_tikka_masala': 22,
 'chikki': 23,
 'daal_baati_churma': 24,
 'daal_puri': 25,
 'dal_makhani': 26,
 'dal_tadka': 27,
 'dharwad_pedha': 28,
 'doodhpak': 29,
 'double_ka_meetha': 30,
 'dum_aloo': 31,
 'gajar_ka_halwa': 32,
 'gavvalu': 33,
 'ghevar': 34,
 'gulab_jamun': 35,
 'imarti': 36,
 'jalebi': 37,
 'kachori': 38,
 'kadai_paneer': 39,
 'kadhi_pakoda': 40,
 'kajjikaya': 41,
 'kakinada_khaja': 42,
 'kalakand': 43,
 'karela_bharta': 44,
 'kofta': 45,
 'kuzhi_paniyaram': 46,
 'lassi': 47,
 'ledikeni': 48,
 'litti_chokha': 49,
 'lyangcha': 50,
 'maach_jhol': 51,
 'makki_di_

In [6]:
class_names = list(train_images.class_indices.keys())
class_names

['adhirasam',
 'aloo_gobi',
 'aloo_matar',
 'aloo_methi',
 'aloo_shimla_mirch',
 'aloo_tikki',
 'anarsa',
 'ariselu',
 'bandar_laddu',
 'basundi',
 'bhatura',
 'bhindi_masala',
 'biryani',
 'boondi',
 'butter_chicken',
 'chak_hao_kheer',
 'cham_cham',
 'chana_masala',
 'chapati',
 'chhena_kheeri',
 'chicken_razala',
 'chicken_tikka',
 'chicken_tikka_masala',
 'chikki',
 'daal_baati_churma',
 'daal_puri',
 'dal_makhani',
 'dal_tadka',
 'dharwad_pedha',
 'doodhpak',
 'double_ka_meetha',
 'dum_aloo',
 'gajar_ka_halwa',
 'gavvalu',
 'ghevar',
 'gulab_jamun',
 'imarti',
 'jalebi',
 'kachori',
 'kadai_paneer',
 'kadhi_pakoda',
 'kajjikaya',
 'kakinada_khaja',
 'kalakand',
 'karela_bharta',
 'kofta',
 'kuzhi_paniyaram',
 'lassi',
 'ledikeni',
 'litti_chokha',
 'lyangcha',
 'maach_jhol',
 'makki_di_roti_sarson_da_saag',
 'malapua',
 'misi_roti',
 'misti_doi',
 'modak',
 'mysore_pak',
 'naan',
 'navrattan_korma',
 'palak_paneer',
 'paneer_butter_masala',
 'phirni',
 'pithe',
 'poha',
 'poornalu

In [7]:
# Image Model
image_input = Input(shape=(128, 128, 3))
x = layers.Conv2D(32, (3, 3), activation='relu')(image_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Flatten()(x)
image_output = layers.Dense(128, activation='relu')(x)

In [8]:
# Classification Layer
final_output = layers.Dense(len(label_encoder.classes_), activation='softmax')(image_output)

In [9]:
# Define the Model
model = models.Model(inputs=image_input, outputs=final_output)

In [10]:
# Train the Model
epochs = 50
model.fit(
    train_images,
    epochs=epochs,
    validation_data=val_images
)

Epoch 1/50
100/100 [==============================] - 77s 763ms/step - loss: 4.4321 - accuracy: 0.0178 - val_loss: 4.2591 - val_accuracy: 0.0350
Epoch 2/50
100/100 [==============================] - 35s 354ms/step - loss: 4.0057 - accuracy: 0.0812 - val_loss: 3.9304 - val_accuracy: 0.0700
Epoch 3/50
100/100 [==============================] - 35s 345ms/step - loss: 2.8102 - accuracy: 0.3303 - val_loss: 3.9348 - val_accuracy: 0.1775
Epoch 4/50
100/100 [==============================] - 33s 334ms/step - loss: 1.1954 - accuracy: 0.7222 - val_loss: 4.4024 - val_accuracy: 0.2412
Epoch 5/50
100/100 [==============================] - 32s 321ms/step - loss: 0.4008 - accuracy: 0.9241 - val_loss: 5.3240 - val_accuracy: 0.2587
Epoch 6/50
100/100 [==============================] - 32s 317ms/step - loss: 0.1687 - accuracy: 0.9753 - val_loss: 5.7840 - val_accuracy: 0.2650
Epoch 7/50
100/100 [==============================] - 30s 297ms/step - loss: 0.0994 - accuracy: 0.9856 - val_loss: 5.6347 - val_ac

In [31]:
# Compile the Model with additional metrics like Precision and Recall
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy', Precision(), Recall()])

In [40]:
history = model.fit(train_images, epochs=10, validation_data=val_images)

# Accessing the history object for training metrics
print(f"Training Accuracy: {history.history['accuracy'][-1]}")
print(f"Validation Accuracy: {history.history['val_accuracy'][-1]}")
print(f"Training Precision: {history.history['precision'][-1]}")
print(f"Validation Precision: {history.history['val_precision'][-1]}")
print(f"Training Recall: {history.history['recall'][-1]}")
print(f"Validation Recall: {history.history['val_recall'][-1]}")


Epoch 1/10


ValueError: in user code:

    File "C:\Users\AKASH\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\AKASH\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\AKASH\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\AKASH\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py", line 1055, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "C:\Users\AKASH\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py", line 1149, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "C:\Users\AKASH\AppData\Roaming\Pyth

In [37]:
import pickle

# Save the trained model
model.save('model.h5')  # Save the Keras model in HDF5 format

# Save the label encoder
with open('model.pkl', 'wb') as encoder_file:
    pickle.dump(label_encoder, encoder_file)

print("model.h5'")
print("model.pkl'")


model.h5'
model.pkl'


In [38]:
# Load the saved model
from tensorflow.keras.models import load_model

loaded_model = load_model('model.h5')
print("Model loaded successfully!")

# Load the saved label encoder
with open('model.pkl', 'rb') as encoder_file:
    loaded_encoder = pickle.load(encoder_file)
print("Label Encoder loaded successfully!")


Model loaded successfully!
Label Encoder loaded successfully!


In [39]:
def predict_food_and_nutrients_with_loaded_model(image_path):
    # Load and preprocess the image
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(128, 128))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict using the loaded model
    prediction = loaded_model.predict(img_array)
    predicted_class = loaded_encoder.inverse_transform([np.argmax(prediction)])[0]

    # Retrieve the nutrients for the predicted food
    nutrients = data[data['Food'] == predicted_class].iloc[:, 1:].to_dict('records')[0]
    return predicted_class, nutrients

# Example Usage
example_image = '33e8639efd.jpg'
predicted_food, predicted_nutrients = predict_food_and_nutrients_with_loaded_model(example_image)
print(f'Predicted Food: {predicted_food}')
print(f'Nutrients: {predicted_nutrients}')


1/1 [==============================] - 0s 59ms/step
Predicted Food: butter_chicken
Nutrients: {'Proteins (g)': 23.0, 'Carbs (g)': 16.5, ' Sugar (g)': 7.0, 'Fats (g)': 14.3}
